### Show ETL results
This code is used to get the results of the metadata database

### Function get data from the database

In [2]:
db_name = "4M"

In [3]:
import psycopg2
import json
import pandas as pd

# --- Load DB config ---
config_file = r"C:\FHNW_lokal\6000\4M\01_ETL\21_load\db_config.json"

def load_config(config_path=config_file):
    with open(config_path) as f:
        return json.load(f)

def run_sql_query_to_dataframe(query: str) -> pd.DataFrame:
    config = load_config()
    try:
        conn = psycopg2.connect(
            database=db_name,
            user=config["user"],
            password=config["password"],
            host=config["host"],
            port=config["port"]
        )
        cur = conn.cursor()
        cur.execute(query)
        rows = cur.fetchall()
        column_names = [desc[0] for desc in cur.description]
        cur.close()
        conn.close()
        return pd.DataFrame(rows, columns=column_names)
    except Exception as e:
        print("Error:", e)
        return pd.DataFrame()

### Number of elements in the database

In [4]:
query = """
SELECT 'merged_dataset_metadata' AS table_name, origin, COUNT(dataset_identifier) AS count
FROM merged_dataset_metadata
GROUP BY origin

UNION ALL

SELECT 'merged_distribution_metadata', origin, COUNT(dataset_identifier)
FROM merged_distribution_metadata
GROUP BY origin

UNION ALL

SELECT 'merged_contact_point_metadata', origin, COUNT(dataset_identifier)
FROM merged_contact_point_metadata
GROUP BY origin;
"""


df = run_sql_query_to_dataframe(query)

# Sort and reset index
df = df.sort_values(['table_name', 'origin']).reset_index(drop=True)

# Compute total per table
totals = df.groupby('table_name')['count'].sum().rename("total").reset_index()

# Merge totals back to original DataFrame
df = df.merge(totals, on='table_name')

# Calculate percentage
df['percent'] = (df['count'] / df['total'] * 100).round(1)

# Optional: reorder columns
df = df[['table_name', 'origin', 'count', 'total', 'percent']]

# Display
from IPython.display import display
print("\nEntry counts per table and origin with total and percentage:\n")
display(df)







Entry counts per table and origin with total and percentage:



,table_name,origin,count,total,percent
0,merged_contact_point_metadata,geocat.ch,13000,26765,48.6
1,merged_contact_point_metadata,opendata.swiss,13765,26765,51.4
2,merged_dataset_metadata,geocat.ch,11511,25055,45.9
3,merged_dataset_metadata,opendata.swiss,13544,25055,54.1
4,merged_distribution_metadata,geocat.ch,15170,70479,21.5
5,merged_distribution_metadata,opendata.swiss,55309,70479,78.5


### Number of descriptions in the database

In [5]:
query = """
SELECT
  -- Dataset counts
  COUNT(dataset_description_DE)                              AS "Dataset DE Filled",
  COUNT(*) - COUNT(dataset_description_DE)                   AS "Dataset DE Empty",
  COUNT(dataset_description_EN)                              AS "Dataset EN Filled",
  COUNT(*) - COUNT(dataset_description_EN)                   AS "Dataset EN Empty",
  COUNT(dataset_description_FR)                              AS "Dataset FR Filled",
  COUNT(*) - COUNT(dataset_description_FR)                   AS "Dataset FR Empty",
  COUNT(dataset_description_IT)                              AS "Dataset IT Filled",
  COUNT(*) - COUNT(dataset_description_IT)                   AS "Dataset IT Empty",
  COUNT(dataset_description_RM)                              AS "Dataset RM Filled",
  COUNT(*) - COUNT(dataset_description_RM)                   AS "Dataset RM Empty",
  COUNT(dataset_description_UNKNOWN)                         AS "Dataset UNKNOWN Filled",
  COUNT(*) - COUNT(dataset_description_UNKNOWN)              AS "Dataset UNKNOWN Empty",
  
  -- Distribution counts
  (SELECT COUNT(distribution_description_DE) FROM merged_distribution_metadata) AS "Distribution DE Filled",
  (SELECT COUNT(*) - COUNT(distribution_description_DE) FROM merged_distribution_metadata) AS "Distribution DE Empty",
  (SELECT COUNT(distribution_description_EN) FROM merged_distribution_metadata) AS "Distribution EN Filled",
  (SELECT COUNT(*) - COUNT(distribution_description_EN) FROM merged_distribution_metadata) AS "Distribution EN Empty",
  (SELECT COUNT(distribution_description_FR) FROM merged_distribution_metadata) AS "Distribution FR Filled",
  (SELECT COUNT(*) - COUNT(distribution_description_FR) FROM merged_distribution_metadata) AS "Distribution FR Empty",
  (SELECT COUNT(distribution_description_IT) FROM merged_distribution_metadata) AS "Distribution IT Filled",
  (SELECT COUNT(*) - COUNT(distribution_description_IT) FROM merged_distribution_metadata) AS "Distribution IT Empty",
  (SELECT COUNT(distribution_description_RM) FROM merged_distribution_metadata) AS "Distribution RM Filled",
  (SELECT COUNT(*) - COUNT(distribution_description_RM) FROM merged_distribution_metadata) AS "Distribution RM Empty",
  (SELECT COUNT(distribution_description_UNKNOWN) FROM merged_distribution_metadata) AS "Distribution UNKNOWN Filled",
  (SELECT COUNT(*) - COUNT(distribution_description_UNKNOWN) FROM merged_distribution_metadata) AS "Distribution UNKNOWN Empty"
FROM merged_dataset_metadata;
"""
from IPython.display import display

# --- Run query and format result ---
df = run_sql_query_to_dataframe(query)

# Transpose and reformat
df_long = df.T.reset_index()
df_long.columns = ['Language + Type', 'Count']
df_long[['Source', 'Language', 'State']] = df_long['Language + Type'].str.extract(r'^(Dataset|Distribution)\s+(\w+)\s+(Filled|Empty)$')

# Pivot into structured table
pivot = df_long.pivot(index='Language', columns=['Source', 'State'], values='Count').fillna(0).astype(int)

# Add percent filled
for source in ['Dataset', 'Distribution']:
    filled = pivot[(source, 'Filled')]
    empty = pivot[(source, 'Empty')]
    pivot[(source, 'Percent Filled')] = ((filled / (filled + empty)) * 100).round(1)

# Reorder languages (optional)
language_order = ['DE', 'EN', 'FR', 'IT', 'RM', 'UNKNOWN']
pivot = pivot.reindex(language_order)

# Display
print("\nDescription counts by language and source with percentages:\n")
display(pivot)





Description counts by language and source with percentages:



Source   Dataset        Distribution               Dataset   Distribution
State     Filled  Empty       Filled  Empty Percent Filled Percent Filled
Language                                                                 
DE         19033   6022        25385  45094           76.0           36.0
EN          4106  20949         6264  64215           16.4            8.9
FR          9246  15809        11086  59393           36.9           15.7
IT          3406  21649         5010  65469           13.6            7.1
RM          1128  23927            4  70475            4.5            0.0
UNKNOWN       20  25035         1279  69200            0.1            1.8

### Analysis descriptions on data portals

In [6]:
query = """
WITH dataset_lang AS (
  SELECT
    origin,
    'Dataset' AS source,
    'DE' AS language,
    COUNT(*) FILTER (WHERE dataset_description_DE IS NOT NULL AND dataset_description_DE <> '') AS filled,
    COUNT(*) FILTER (WHERE dataset_description_DE IS NULL OR dataset_description_DE = '') AS empty
  FROM merged_dataset_metadata
  GROUP BY origin

  UNION ALL

  SELECT origin, 'Dataset', 'EN', COUNT(*) FILTER (WHERE dataset_description_EN <> '' AND dataset_description_EN IS NOT NULL),
                                 COUNT(*) FILTER (WHERE dataset_description_EN = '' OR dataset_description_EN IS NULL)
  FROM merged_dataset_metadata GROUP BY origin

  UNION ALL

  SELECT origin, 'Dataset', 'FR', COUNT(*) FILTER (WHERE dataset_description_FR <> '' AND dataset_description_FR IS NOT NULL),
                                 COUNT(*) FILTER (WHERE dataset_description_FR = '' OR dataset_description_FR IS NULL)
  FROM merged_dataset_metadata GROUP BY origin

  UNION ALL

  SELECT origin, 'Dataset', 'IT', COUNT(*) FILTER (WHERE dataset_description_IT <> '' AND dataset_description_IT IS NOT NULL),
                                 COUNT(*) FILTER (WHERE dataset_description_IT = '' OR dataset_description_IT IS NULL)
  FROM merged_dataset_metadata GROUP BY origin

  UNION ALL

  SELECT origin, 'Dataset', 'RM', COUNT(*) FILTER (WHERE dataset_description_RM <> '' AND dataset_description_RM IS NOT NULL),
                                 COUNT(*) FILTER (WHERE dataset_description_RM = '' OR dataset_description_RM IS NULL)
  FROM merged_dataset_metadata GROUP BY origin

  UNION ALL

  SELECT origin, 'Dataset', 'UNKNOWN', COUNT(*) FILTER (WHERE dataset_description_UNKNOWN <> '' AND dataset_description_UNKNOWN IS NOT NULL),
                                       COUNT(*) FILTER (WHERE dataset_description_UNKNOWN = '' OR dataset_description_UNKNOWN IS NULL)
  FROM merged_dataset_metadata GROUP BY origin
),
distribution_lang AS (
  SELECT
    origin,
    'Distribution' AS source,
    'DE' AS language,
    COUNT(*) FILTER (WHERE distribution_description_DE IS NOT NULL AND distribution_description_DE <> '') AS filled,
    COUNT(*) FILTER (WHERE distribution_description_DE IS NULL OR distribution_description_DE = '') AS empty
  FROM merged_distribution_metadata
  GROUP BY origin

  UNION ALL

  SELECT origin, 'Distribution', 'EN', COUNT(*) FILTER (WHERE distribution_description_EN <> '' AND distribution_description_EN IS NOT NULL),
                                         COUNT(*) FILTER (WHERE distribution_description_EN = '' OR distribution_description_EN IS NULL)
  FROM merged_distribution_metadata GROUP BY origin

  UNION ALL

  SELECT origin, 'Distribution', 'FR', COUNT(*) FILTER (WHERE distribution_description_FR <> '' AND distribution_description_FR IS NOT NULL),
                                         COUNT(*) FILTER (WHERE distribution_description_FR = '' OR distribution_description_FR IS NULL)
  FROM merged_distribution_metadata GROUP BY origin

  UNION ALL

  SELECT origin, 'Distribution', 'IT', COUNT(*) FILTER (WHERE distribution_description_IT <> '' AND distribution_description_IT IS NOT NULL),
                                         COUNT(*) FILTER (WHERE distribution_description_IT = '' OR distribution_description_IT IS NULL)
  FROM merged_distribution_metadata GROUP BY origin

  UNION ALL

  SELECT origin, 'Distribution', 'RM', COUNT(*) FILTER (WHERE distribution_description_RM <> '' AND distribution_description_RM IS NOT NULL),
                                         COUNT(*) FILTER (WHERE distribution_description_RM = '' OR distribution_description_RM IS NULL)
  FROM merged_distribution_metadata GROUP BY origin

  UNION ALL

  SELECT origin, 'Distribution', 'UNKNOWN', COUNT(*) FILTER (WHERE distribution_description_UNKNOWN <> '' AND distribution_description_UNKNOWN IS NOT NULL),
                                             COUNT(*) FILTER (WHERE distribution_description_UNKNOWN = '' OR distribution_description_UNKNOWN IS NULL)
  FROM merged_distribution_metadata GROUP BY origin
)

SELECT * FROM dataset_lang
UNION ALL
SELECT * FROM distribution_lang;
"""

df = run_sql_query_to_dataframe(query)

# Berechne Prozent gefüllt
df['percent_filled'] = (df['filled'] / (df['filled'] + df['empty']) * 100).round(1)

# Optional: sortieren
df = df.sort_values(['origin', 'source', 'language'])

# Gruppiert anzeigen (z. B. eine Tabelle pro Portal)
for portal, group in df.groupby('origin'):
    print(f"\nDescription completeness for portal: {portal}\n")
    display(group.pivot(index='language', columns='source', values=['filled', 'empty', 'percent_filled']))




Description completeness for portal: geocat.ch



filled                 empty              percent_filled  \
source   Dataset Distribution  Dataset Distribution        Dataset   
language                                                             
DE        8750.0       6198.0   2761.0       8972.0           76.0   
EN        1122.0        610.0  10389.0      14560.0            9.7   
FR        4488.0       3026.0   7023.0      12144.0           39.0   
IT         820.0         45.0  10691.0      15125.0            7.1   
RM         252.0          0.0  11259.0      15170.0            2.2   
UNKNOWN     20.0       1279.0  11491.0      13891.0            0.2   

                       
source   Distribution  
language               
DE               40.9  
EN                4.0  
FR               19.9  
IT                0.3  
RM                0.0  
UNKNOWN           8.4


Description completeness for portal: opendata.swiss



filled                 empty              percent_filled  \
source    Dataset Distribution  Dataset Distribution        Dataset   
language                                                              
DE        10283.0      19187.0   3261.0      36122.0           75.9   
EN         2984.0       5654.0  10560.0      49655.0           22.0   
FR         4758.0       8060.0   8786.0      47249.0           35.1   
IT         2586.0       4965.0  10958.0      50344.0           19.1   
RM          876.0          4.0  12668.0      55305.0            6.5   
UNKNOWN       0.0          0.0  13544.0      55309.0            0.0   

                       
source   Distribution  
language               
DE               34.7  
EN               10.2  
FR               14.6  
IT                9.0  
RM                0.0  
UNKNOWN           0.0

In [7]:
query = """
SELECT
  -- Dataset title counts
  COUNT(*) FILTER (WHERE dataset_title_DE IS NOT NULL AND dataset_title_DE <> '') AS "Dataset DE Filled",
  COUNT(*) FILTER (WHERE dataset_title_DE IS NULL OR dataset_title_DE = '')       AS "Dataset DE Empty",

  COUNT(*) FILTER (WHERE dataset_title_EN IS NOT NULL AND dataset_title_EN <> '') AS "Dataset EN Filled",
  COUNT(*) FILTER (WHERE dataset_title_EN IS NULL OR dataset_title_EN = '')       AS "Dataset EN Empty",

  COUNT(*) FILTER (WHERE dataset_title_FR IS NOT NULL AND dataset_title_FR <> '') AS "Dataset FR Filled",
  COUNT(*) FILTER (WHERE dataset_title_FR IS NULL OR dataset_title_FR = '')       AS "Dataset FR Empty",

  COUNT(*) FILTER (WHERE dataset_title_IT IS NOT NULL AND dataset_title_IT <> '') AS "Dataset IT Filled",
  COUNT(*) FILTER (WHERE dataset_title_IT IS NULL OR dataset_title_IT = '')       AS "Dataset IT Empty",

  COUNT(*) FILTER (WHERE dataset_title_RM IS NOT NULL AND dataset_title_RM <> '') AS "Dataset RM Filled",
  COUNT(*) FILTER (WHERE dataset_title_RM IS NULL OR dataset_title_RM = '')       AS "Dataset RM Empty",

  COUNT(*) FILTER (WHERE dataset_title_UNKNOWN IS NOT NULL AND dataset_title_UNKNOWN <> '') AS "Dataset UNKNOWN Filled",
  COUNT(*) FILTER (WHERE dataset_title_UNKNOWN IS NULL OR dataset_title_UNKNOWN = '')       AS "Dataset UNKNOWN Empty",

  -- Distribution title counts
  (SELECT COUNT(*) FILTER (WHERE distribution_title_DE IS NOT NULL AND distribution_title_DE <> '') FROM merged_distribution_metadata) AS "Distribution DE Filled",
  (SELECT COUNT(*) FILTER (WHERE distribution_title_DE IS NULL OR distribution_title_DE = '') FROM merged_distribution_metadata)       AS "Distribution DE Empty",

  (SELECT COUNT(*) FILTER (WHERE distribution_title_EN IS NOT NULL AND distribution_title_EN <> '') FROM merged_distribution_metadata) AS "Distribution EN Filled",
  (SELECT COUNT(*) FILTER (WHERE distribution_title_EN IS NULL OR distribution_title_EN = '') FROM merged_distribution_metadata)       AS "Distribution EN Empty",

  (SELECT COUNT(*) FILTER (WHERE distribution_title_FR IS NOT NULL AND distribution_title_FR <> '') FROM merged_distribution_metadata) AS "Distribution FR Filled",
  (SELECT COUNT(*) FILTER (WHERE distribution_title_FR IS NULL OR distribution_title_FR = '') FROM merged_distribution_metadata)       AS "Distribution FR Empty",

  (SELECT COUNT(*) FILTER (WHERE distribution_title_IT IS NOT NULL AND distribution_title_IT <> '') FROM merged_distribution_metadata) AS "Distribution IT Filled",
  (SELECT COUNT(*) FILTER (WHERE distribution_title_IT IS NULL OR distribution_title_IT = '') FROM merged_distribution_metadata)       AS "Distribution IT Empty",

  (SELECT COUNT(*) FILTER (WHERE distribution_title_RM IS NOT NULL AND distribution_title_RM <> '') FROM merged_distribution_metadata) AS "Distribution RM Filled",
  (SELECT COUNT(*) FILTER (WHERE distribution_title_RM IS NULL OR distribution_title_RM = '') FROM merged_distribution_metadata)       AS "Distribution RM Empty",

  (SELECT COUNT(*) FILTER (WHERE distribution_title_UNKNOWN IS NOT NULL AND distribution_title_UNKNOWN <> '') FROM merged_distribution_metadata) AS "Distribution UNKNOWN Filled",
  (SELECT COUNT(*) FILTER (WHERE distribution_title_UNKNOWN IS NULL OR distribution_title_UNKNOWN = '') FROM merged_distribution_metadata)       AS "Distribution UNKNOWN Empty"
FROM merged_dataset_metadata;
"""
from IPython.display import display

df = run_sql_query_to_dataframe(query)

# Transpose and reformat
df_long = df.T.reset_index()
df_long.columns = ['Language + Type', 'Count']
df_long[['Source', 'Language', 'State']] = df_long['Language + Type'].str.extract(r'^(Dataset|Distribution)\s+(\w+)\s+(Filled|Empty)$')

# Pivot into structured table
pivot = df_long.pivot(index='Language', columns=['Source', 'State'], values='Count').fillna(0).astype(int)

# Add percent filled
for source in ['Dataset', 'Distribution']:
    filled = pivot[(source, 'Filled')]
    empty = pivot[(source, 'Empty')]
    pivot[(source, 'Percent Filled')] = ((filled / (filled + empty)) * 100).round(1)

# Reorder languages (optional)
language_order = ['DE', 'EN', 'FR', 'IT', 'RM', 'UNKNOWN']
pivot = pivot.reindex(language_order)

# Display
print("\nTitle counts by language and source with percentages:\n")
display(pivot)



Title counts by language and source with percentages:



Source   Dataset        Distribution               Dataset   Distribution
State     Filled  Empty       Filled  Empty Percent Filled Percent Filled
Language                                                                 
DE         21217   3838        38447  32032           84.7           54.6
EN          4751  20304        13185  57294           19.0           18.7
FR         11162  13893        16446  54033           44.5           23.3
IT          4480  20575         8983  61496           17.9           12.7
RM          1145  23910         1976  68503            4.6            2.8
UNKNOWN      106  24949          985  69494            0.4            1.4

### Analysis of how often keywords are present

In [8]:
query = """
SELECT
  -- DE
  COUNT(*) FILTER (WHERE dataset_keyword_DE IS NOT NULL AND dataset_keyword_DE <> '')      AS "DE Filled",
  COUNT(*) FILTER (WHERE dataset_keyword_DE IS NULL OR dataset_keyword_DE = '')            AS "DE Empty",
  SUM(array_length(string_to_array(dataset_keyword_DE, ','), 1))                           AS "DE Total Keywords count",

  -- EN
  COUNT(*) FILTER (WHERE dataset_keyword_EN IS NOT NULL AND dataset_keyword_EN <> '')      AS "EN Filled",
  COUNT(*) FILTER (WHERE dataset_keyword_EN IS NULL OR dataset_keyword_EN = '')            AS "EN Empty",
  SUM(array_length(string_to_array(dataset_keyword_EN, ','), 1))                           AS "EN Total Keywords count",

  -- FR
  COUNT(*) FILTER (WHERE dataset_keyword_FR IS NOT NULL AND dataset_keyword_FR <> '')      AS "FR Filled",
  COUNT(*) FILTER (WHERE dataset_keyword_FR IS NULL OR dataset_keyword_FR = '')            AS "FR Empty",
  SUM(array_length(string_to_array(dataset_keyword_FR, ','), 1))                           AS "FR Total Keywords count",

  -- IT
  COUNT(*) FILTER (WHERE dataset_keyword_IT IS NOT NULL AND dataset_keyword_IT <> '')      AS "IT Filled",
  COUNT(*) FILTER (WHERE dataset_keyword_IT IS NULL OR dataset_keyword_IT = '')            AS "IT Empty",
  SUM(array_length(string_to_array(dataset_keyword_IT, ','), 1))                           AS "IT Total Keywords count",

  -- RM
  COUNT(*) FILTER (WHERE dataset_keyword_RM IS NOT NULL AND dataset_keyword_RM <> '')      AS "RM Filled",
  COUNT(*) FILTER (WHERE dataset_keyword_RM IS NULL OR dataset_keyword_RM = '')            AS "RM Empty",
  SUM(array_length(string_to_array(dataset_keyword_RM, ','), 1))                           AS "RM Total Keywords count",

  -- UNKNOWN
  COUNT(*) FILTER (WHERE dataset_keyword_UNKNOWN IS NOT NULL AND dataset_keyword_UNKNOWN <> '') AS "UNKNOWN Filled",
  COUNT(*) FILTER (WHERE dataset_keyword_UNKNOWN IS NULL OR dataset_keyword_UNKNOWN = '')       AS "UNKNOWN Empty",
  SUM(array_length(string_to_array(dataset_keyword_UNKNOWN, ','), 1))                        AS "UNKNOWN Total Keywords count"
FROM merged_dataset_metadata;
"""

df = run_sql_query_to_dataframe(query)

# Umstrukturieren
df_long = df.T.reset_index()
df_long.columns = ['Metric', 'Count']
df_long[['Language', 'Type']] = df_long['Metric'].str.extract(r'^(.*?)\s+(Filled|Empty|Total Keywords count)$')

# Pivot: jede Sprache als Zeile, jede Metrik als Spalte
final_table = df_long.pivot(index='Language', columns='Type', values='Count').fillna(0).astype(int)

# Prozentwert berechnen
final_table['Percent Filled'] = (
    final_table['Filled'] / (final_table['Filled'] + final_table['Empty']) * 100
).round(1)

# Total der Keywords (ergänzend, umbenannt für Klarheit)
final_table['Total Keywords'] = final_table['Total Keywords count']

# Unnötige Spalte entfernen (optional)
final_table = final_table.drop(columns='Total Keywords count', errors='ignore')

# Sprachreihenfolge (optional)
language_order = ['DE', 'EN', 'FR', 'IT', 'RM', 'UNKNOWN']
final_table = final_table.reindex(language_order)

# Summenzeile berechnen
total_row = final_table.sum(numeric_only=True)
total_row.name = 'TOTAL'

# Prozentwert für TOTAL berechnen (optional, sinnvoll nur wenn alle Sprachen gleichwertig behandelt werden sollen)
total_row['Percent Filled'] = (
    total_row['Filled'] / (total_row['Filled'] + total_row['Empty']) * 100
).round(1)

# Summenzeile anhängen
final_table = final_table.append(total_row)


# Ergebnis anzeigen
print("\nKeyword summary per language (dataset):\n")
display(final_table)





Keyword summary per language (dataset):



C:\Temp\QGIS\ipykernel_14720\3884528887.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_table = final_table.append(total_row)


Type,Empty,Filled,Percent Filled,Total Keywords
Language,,,,
DE,8017.0,17038.0,68.0,66043.0
EN,12726.0,12329.0,49.2,42870.0
FR,11985.0,13070.0,52.2,45282.0
IT,13044.0,12011.0,47.9,40067.0
RM,25055.0,0.0,0.0,0.0
UNKNOWN,18176.0,6879.0,27.5,25462.0
TOTAL,89003.0,61327.0,40.8,219724.0


### Number of empty descriptions and titles by data platform

In [9]:
query = """
SELECT
  COUNT(*) AS "Total",
  COUNT(*) FILTER (WHERE distribution_download_url IS NOT NULL AND distribution_download_url <> '') AS "Filled",
  COUNT(*) FILTER (WHERE distribution_download_url IS NULL OR distribution_download_url = '') AS "Empty",
  COUNT(DISTINCT distribution_download_url) AS "Distinct"
FROM merged_distribution_metadata;
"""

df = run_sql_query_to_dataframe(query)

# Umstrukturieren
df_long = df.T.reset_index()
df_long.columns = ['Metric', 'Value']
df_long = df_long.set_index('Metric')

# Prozentwerte berechnen
total = df_long.loc['Total', 'Value']
filled = df_long.loc['Filled', 'Value']
empty = df_long.loc['Empty', 'Value']

df_long.loc['Filled (%)'] = round(filled / total * 100, 1)
df_long.loc['Empty (%)'] = round(empty / total * 100, 1)

# Ergebnis anzeigen
print("\nDownload URL summary from distribution table (with percentages):\n")
display(df_long)



Download URL summary from distribution table (with percentages):



,Value
Metric,
Total,70479.0
Filled,40222.0
Empty,30257.0
Distinct,33059.0
Filled (%),57.1
Empty (%),42.9


### Examples of download URL that are general

In [10]:
query = """
SELECT distribution_download_url, COUNT(*) AS occurrences
FROM merged_distribution_metadata
WHERE distribution_download_url IS NOT NULL AND distribution_download_url <> ''
GROUP BY distribution_download_url
HAVING COUNT(*) > 1
ORDER BY occurrences DESC
LIMIT 10;
"""

count_mobility = run_sql_query_to_dataframe(query)

print("\nTop 10 duplicate distribution_download_url entries:\n")
display(count_mobility)



Top 10 duplicate distribution_download_url entries:



,distribution_download_url,occurrences
0,https://geodienste.ch/db/av_0/deu?SERVICE=WMS&...,217
1,https://geoshop.lisag.ch/client5/NWgeoshop.igs...,188
2,https://arp-daten.bl.ch/arpdaten/arp_zustaendi...,174
3,https://www.baselland.ch/politik-und-behorden/...,172
4,https://map.geo.tg.ch,156
5,https://wms.geo.admin.ch/?SERVICE=WMS&VERSION=...,152
6,https://www.gis-daten.ch/map/plan_fuer_grundbu...,127
7,https://sdi.georhena.eu/geoserver/ows?SERVICE=...,124
8,http://shop.geo.bs.ch/,109
9,https://wmts.geo.admin.ch/EPSG/3857/1.0.0/WMTS...,106


### Number of mobility data

In [11]:
query = """
SELECT dataset_is_mobility, COUNT(*) AS occurrences
FROM merged_dataset_metadata
GROUP BY dataset_is_mobility
ORDER BY occurrences DESC;
"""

count_mobility = run_sql_query_to_dataframe(query)

print("\nTop 10 duplicate distribution_download_url entries:\n")
display(count_mobility)



Top 10 duplicate distribution_download_url entries:



,dataset_is_mobility,occurrences
0,False,21416
1,True,3639


### Number of dataset themes that are filled

In [12]:
query = """
SELECT
  COALESCE(origin, 'TOTAL') AS origin,
  COUNT(*) FILTER (WHERE dataset_theme IS NOT NULL AND dataset_theme <> '') AS theme_filled,
  COUNT(*) FILTER (WHERE dataset_theme IS NULL OR dataset_theme = '')       AS theme_empty,
  COUNT(*)                                                                 AS total
FROM merged_dataset_metadata
GROUP BY ROLLUP(origin)
ORDER BY origin NULLS LAST;

"""


df = run_sql_query_to_dataframe(query)

# Prozent-Spalte berechnen
df['percent_filled'] = (df['theme_filled'] / df['total'] * 100).round(1)

# Optional: TOTAL-Zeile ans Ende sortieren
df = df.sort_values(by='origin', key=lambda x: x == 'TOTAL').reset_index(drop=True)

display(df)



,origin,theme_filled,theme_empty,total,percent_filled
0,geocat.ch,11274,237,11511,97.9
1,opendata.swiss,11933,1611,13544,88.1
2,TOTAL,23207,1848,25055,92.6


### Frequency of the different topics

In [13]:
query = """
SELECT
  dataset_theme,
  COUNT(*) AS count
FROM merged_dataset_metadata
WHERE dataset_theme IS NOT NULL AND TRIM(dataset_theme) <> ''
GROUP BY dataset_theme
ORDER BY count DESC;
"""

df = run_sql_query_to_dataframe(query)

df['dataset_theme'] = df['dataset_theme'].str.replace(
    "http://publications.europa.eu/resource/authority/data-theme/", "", regex=False
)



display(df)


,dataset_theme,count
0,ENVI,3270
1,GOVE,1658
2,EDUC,1617
3,SOCI,1494
4,ECON,1225
...,...,...
616,"['geoscientificInformation', 'geoscientificInf...",1
617,"['society', 'intelligenceMilitary']",1
618,"['imageryBaseMapsEarthCover', 'imageryBaseMaps...",1
619,"['imageryBaseMapsEarthCover', 'imageryBaseMaps...",1


### Number of spatial datasets that are filled

In [14]:
query = """
SELECT
  origin,
  COUNT(*) FILTER (
    WHERE dataset_spatial IS NOT NULL AND dataset_spatial <> 'not_found'
  ) AS spatial_filled,
  COUNT(*) FILTER (
    WHERE dataset_spatial IS NULL OR dataset_spatial = 'not_found'
  ) AS spatial_empty,
  COUNT(*) AS total
FROM merged_dataset_metadata
GROUP BY origin
ORDER BY origin;

"""


df = run_sql_query_to_dataframe(query)

# Prozent gefüllter Theme-Spalten
df['percent_filled'] = (df['spatial_filled'] / df['total'] * 100).round(1)

# Optional: Sortieren
df = df.sort_values('total', ascending=False)

display(df)


,origin,spatial_filled,spatial_empty,total,percent_filled
1,opendata.swiss,7326,6218,13544,54.1
0,geocat.ch,9422,2089,11511,81.9


### Number of dataset locations that are filled

In [15]:
query = """SELECT
  origin,
  COUNT(*) FILTER (
    WHERE dataset_location IS NOT NULL AND dataset_location <> 'not_found'
  ) AS location_filled,
  COUNT(*) FILTER (
    WHERE dataset_location IS NULL OR dataset_location = 'not_found'
  ) AS location_empty,
  COUNT(*) AS total
FROM merged_dataset_metadata
GROUP BY origin
ORDER BY origin;

"""


df = run_sql_query_to_dataframe(query)

# Prozent gefüllter Theme-Spalten
df['percent_filled'] = (df['location_filled'] / df['total'] * 100).round(1)

# Optional: Sortieren
df = df.sort_values('total', ascending=False)

display(df)


,origin,location_filled,location_empty,total,percent_filled
1,opendata.swiss,6029,7515,13544,44.5
0,geocat.ch,7800,3711,11511,67.8


### Location layout available

In [16]:
query = """SELECT
  COUNT(*) AS total,

  -- 1. Beide gefüllt
  COUNT(*) FILTER (
    WHERE TRIM(COALESCE(dataset_location, '')) <> ''
      AND TRIM(COALESCE(dataset_location, '')) <> 'not_found'
      AND TRIM(COALESCE(dataset_spatial, '')) <> ''
      AND TRIM(COALESCE(dataset_spatial, '')) <> 'not_found'
  ) AS both_filled,

  -- 2. Nur location gefüllt
  COUNT(*) FILTER (
    WHERE TRIM(COALESCE(dataset_location, '')) <> ''
      AND TRIM(COALESCE(dataset_location, '')) <> 'not_found'
      AND (dataset_spatial IS NULL OR TRIM(dataset_spatial) = '' OR TRIM(dataset_spatial) = 'not_found')
  ) AS location_only,

  -- 3. Nur spatial gefüllt
  COUNT(*) FILTER (
    WHERE TRIM(COALESCE(dataset_spatial, '')) <> ''
      AND TRIM(COALESCE(dataset_spatial, '')) <> 'not_found'
  ) AS spatial_only,

  -- 4. Beide leer
  COUNT(*) FILTER (
    WHERE (dataset_location IS NULL OR TRIM(dataset_location) = '' OR TRIM(dataset_location) = 'not_found')
      AND (dataset_spatial IS NULL OR TRIM(dataset_spatial) = '' OR TRIM(dataset_spatial) = 'not_found')
  ) AS both_empty
FROM merged_dataset_metadata;

"""

df_total = run_sql_query_to_dataframe(query)

# Prozentspalten berechnen
total = df_total.at[0, 'total']
df_total['% both_filled'] = (df_total['both_filled'] / total * 100).round(1)
df_total['% location_only'] = (df_total['location_only'] / total * 100).round(1)
df_total['% spatial_only'] = (df_total['spatial_only'] / total * 100).round(1)
df_total['% both_empty'] = (df_total['both_empty'] / total * 100).round(1)

display(df_total)


,total,both_filled,location_only,spatial_only,both_empty,% both_filled,% location_only,% spatial_only,% both_empty
0,25055,10584,3245,16748,5062,42.2,13.0,66.8,20.2


### Location allocation available separated by portals

In [17]:


query = """SELECT
  origin,

  -- Zähle alle Fälle
  COUNT(*) AS total,

  -- 1. Beide gefüllt
  COUNT(*) FILTER (
    WHERE TRIM(COALESCE(dataset_location, '')) <> ''
      AND TRIM(COALESCE(dataset_location, '')) <> 'not_found'
      AND TRIM(COALESCE(dataset_spatial, '')) <> ''
      AND TRIM(COALESCE(dataset_spatial, '')) <> 'not_found'
  ) AS both_filled,

  -- 2. Nur location gefüllt
  COUNT(*) FILTER (
    WHERE TRIM(COALESCE(dataset_location, '')) <> ''
      AND TRIM(COALESCE(dataset_location, '')) <> 'not_found'
      AND (
        dataset_spatial IS NULL
        OR TRIM(dataset_spatial) = ''
        OR TRIM(dataset_spatial) = 'not_found'
      )
  ) AS location_only,

  -- 3. Nur spatial gefüllt
  COUNT(*) FILTER (
    WHERE TRIM(COALESCE(dataset_spatial, '')) <> ''
      AND TRIM(COALESCE(dataset_spatial, '')) <> 'not_found'
      AND (
        dataset_location IS NULL
        OR TRIM(dataset_location) = ''
        OR TRIM(dataset_location) = 'not_found'
      )
  ) AS spatial_only,

  -- 4. Beide leer
  COUNT(*) FILTER (
    WHERE (
      dataset_location IS NULL
      OR TRIM(dataset_location) = ''
      OR TRIM(dataset_location) = 'not_found'
    )
    AND (
      dataset_spatial IS NULL
      OR TRIM(dataset_spatial) = ''
      OR TRIM(dataset_spatial) = 'not_found'
    )
  ) AS both_empty

FROM merged_dataset_metadata
GROUP BY origin
ORDER BY origin;
"""
df = run_sql_query_to_dataframe(query)

# Prozentwerte berechnen
df['% both_filled'] = (df['both_filled'] / df['total'] * 100).round(1)
df['% location_only'] = (df['location_only'] / df['total'] * 100).round(1)
df['% spatial_only'] = (df['spatial_only'] / df['total'] * 100).round(1)
df['% both_empty'] = (df['both_empty'] / df['total'] * 100).round(1)

# Ergebnis anzeigen
display(df)




,origin,total,both_filled,location_only,spatial_only,both_empty,% both_filled,% location_only,% spatial_only,% both_empty
0,geocat.ch,11511,7483,317,1939,1772,65.0,2.8,16.8,15.4
1,opendata.swiss,13544,3101,2928,4225,3290,22.9,21.6,31.2,24.3


### Distribution of language identification results

In [32]:
query = """
SELECT 
  dataset_language_quality,
  COUNT(*) AS anzahl,
  ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 3) AS prozent
FROM 
  merged_dataset_metadata
GROUP BY 
  dataset_language_quality
ORDER BY 
  prozent DESC;

"""

df = run_sql_query_to_dataframe(query)

display(df)


,dataset_language_quality,anzahl,prozent
0,correct,10938,43.656
1,identical_description,7339,29.292
2,incorrect,6778,27.052


### Median length of descriptions by language

In [33]:
query = """
SELECT
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY dataset_description_length_DE) FILTER (WHERE dataset_description_length_DE IS NOT NULL) AS median_description_length_DE,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY dataset_description_length_EN) FILTER (WHERE dataset_description_length_EN IS NOT NULL) AS median_description_length_EN,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY dataset_description_length_FR) FILTER (WHERE dataset_description_length_FR IS NOT NULL) AS median_description_length_FR,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY dataset_description_length_IT) FILTER (WHERE dataset_description_length_IT IS NOT NULL) AS median_description_length_IT,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY dataset_description_length_RM) FILTER (WHERE dataset_description_length_RM IS NOT NULL) AS median_description_length_RM
FROM
    merged_dataset_metadata;
"""

df = run_sql_query_to_dataframe(query)
display(df)

,median_description_length_de,median_description_length_en,median_description_length_fr,median_description_length_it,median_description_length_rm
0,241.0,361.5,380.5,404.5,212.0


### Median length of descriptions by language per data publisher

In [34]:
query = """
SELECT
    dataset_publisher_name,
    COUNT(*) AS total_entries_for_publisher, -- Fügt die Anzahl der Einträge pro Publisher hinzu
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY dataset_description_length_DE) FILTER (WHERE dataset_description_length_DE IS NOT NULL) AS median_description_length_DE,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY dataset_description_length_EN) FILTER (WHERE dataset_description_length_EN IS NOT NULL) AS median_description_length_EN,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY dataset_description_length_FR) FILTER (WHERE dataset_description_length_FR IS NOT NULL) AS median_description_length_FR,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY dataset_description_length_IT) FILTER (WHERE dataset_description_length_IT IS NOT NULL) AS median_description_length_IT,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY dataset_description_length_RM) FILTER (WHERE dataset_description_length_RM IS NOT NULL) AS median_description_length_RM
FROM
    merged_dataset_metadata
GROUP BY
    dataset_publisher_name
ORDER BY
    total_entries_for_publisher DESC, -- Sortiert primär nach der Anzahl der Einträge (absteigend)
    dataset_publisher_name;           -- Sortiert sekundär nach Publisher-Name (aufsteigend)
"""

df = run_sql_query_to_dataframe(query)
display(df)

,dataset_publisher_name,total_entries_for_publisher,median_description_length_de,median_description_length_en,median_description_length_fr,median_description_length_it,median_description_length_rm
0,None,13548,348.0,454.5,560.0,490.0,199.0
1,Bundesamt für Landestopografie swisstopo,1003,154.0,513.0,180.0,589.0,576.0
2,Amt für Raumentwicklung und Geoinformation SG,911,148.0,NaN,NaN,NaN,NaN
3,Amt für Geoinformation BL,766,398.0,NaN,NaN,NaN,NaN
4,Département du territoire,541,NaN,NaN,1691.0,NaN,NaN
...,...,...,...,...,...,...,...
569,Werdenberger Namenbuch,1,33.0,NaN,NaN,NaN,NaN
570,"Widrig, Leumann & Willi AG",1,67.0,NaN,NaN,NaN,NaN
571,Zentralbibliothek Zürich,1,294.0,339.0,350.0,364.0,NaN
572,Zweifel AG,1,270.0,NaN,NaN,NaN,NaN


### Distribution of HTTP status codes

In [37]:
query = """
SELECT
    distribution_download_url_status_code,
    COUNT(*) AS anzahl,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 1) AS prozent
FROM
    merged_distribution_metadata
WHERE
    distribution_download_url_status_code <> 0 -- Filtert Status-Codes, die nicht 0 sind
GROUP BY
    distribution_download_url_status_code
ORDER BY
    prozent DESC;

"""

df = run_sql_query_to_dataframe(query)

display(df)


,distribution_download_url_status_code,anzahl,prozent
0,200,32720,84.7
1,405,2006,5.2
2,403,1912,5.0
3,404,981,2.5
4,400,545,1.4
5,204,229,0.6
6,503,103,0.3
7,500,84,0.2
8,401,18,0.0
9,499,17,0.0


### Distribution of formate names in %

In [ ]:
query = """
SELECT
    distribution_format_name,
    COUNT(*) AS anzahl,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 1) AS prozent
FROM
    merged_distribution_metadata
WHERE
    distribution_format_name <> Null -- Filtert Status-Codes, die nicht 0 sind
GROUP BY
    distribution_format_name
ORDER BY
    prozent DESC;

"""

df = run_sql_query_to_dataframe(query)

display(df)
